<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021)

In [1]:
import requests
import pandas as pd
import numpy as np
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## Day 3: Binary Diagnostic

In [2]:
example = """00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010""".split()

In [5]:
matrix = [list(n) for n in example]
matrix

[['0', '0', '1', '0', '0'],
 ['1', '1', '1', '1', '0'],
 ['1', '0', '1', '1', '0'],
 ['1', '0', '1', '1', '1'],
 ['1', '0', '1', '0', '1'],
 ['0', '1', '1', '1', '1'],
 ['0', '0', '1', '1', '1'],
 ['1', '1', '1', '0', '0'],
 ['1', '0', '0', '0', '0'],
 ['1', '1', '0', '0', '1'],
 ['0', '0', '0', '1', '0'],
 ['0', '1', '0', '1', '0']]

In [52]:
[str(i) for i in range(len(matrix[0]))]

['0', '1', '2', '3', '4']

In [55]:
df = pd.DataFrame(matrix).astype(int)
df

,0,1,2,3,4
0,0,0,1,0,0
1,1,1,1,1,0
2,1,0,1,1,0
3,1,0,1,1,1
4,1,0,1,0,1
5,0,1,1,1,1
6,0,0,1,1,1
7,1,1,1,0,0
8,1,0,0,0,0
9,1,1,0,0,1


In [70]:
for i in range(df.shape[1]):
  vc = df[i].value_counts().to_dict()
  print(vc, vc[0] > vc[1])

{1: 7, 0: 5} False
{0: 7, 1: 5} True
{1: 8, 0: 4} False
{1: 7, 0: 5} False
{0: 7, 1: 5} True


In [69]:
vc[0]

7

In [30]:
def bin_to_int(list):
  return int('0b'+"".join(np.array(list).astype(str)), base=2)

def solve_part1(data):
  matrix = [list(n) for n in data]
  rates = [(round(np.mean(line)), round(1-np.mean(line))) for line in np.array(matrix).T.astype(int)]
  rates_int = [bin_to_int(l) for l in np.array(rates).T]
  return np.multiply(*rates_int)

In [31]:
solve_part1(example)

198

In [32]:
r = requests.get(path+'AoC2021_03.txt')
data = r.text.split('\n')[:-1]
solve_part1(data)

3813416

## Day 2: Dive!

In [ ]:
example = """forward 5
down 5
forward 8
up 3
down 8
forward 2""".split("\n")

In [ ]:
def get_hp(row):
  if row['direction'] == 'forward':
    return row['value']
  else:
    return 0

def get_depth(row):
  if row['direction'] == 'down':
    return row['value']
  elif row['direction'] == 'up':
    return -row['value']
  else:
    return 0

def solve_part1(data):
  """Calculate a horizontal position and depth"""

  df = pd.DataFrame(data, columns=['instruction'])
  df['direction'] = df.instruction.apply(lambda s: s.split()[0])
  df['value'] = df.instruction.apply(lambda s: int(s.split()[1]))
  df['hp'] = df.apply(get_hp, axis=1)
  df['depth'] = df.apply(get_depth, axis=1)
  result = df.hp.sum()*df.depth.sum()
  return result

In [ ]:
solve_part1(example)

150

In [ ]:
r = requests.get(path+'AoC2021_02.txt')
data = r.text.split('\n')[:-1]
solve_part1(data)

2036120

In [ ]:
def solve_part2(data):
  """Calculate a horizontal position and depth with aim"""

  df = pd.DataFrame(data, columns=['instruction'])
  df['direction'] = df.instruction.apply(lambda s: s.split()[0])
  df['value'] = df.instruction.apply(lambda s: int(s.split()[1]))
  hp, depth, aim = 0, 0, 0
  for _, row in df.iterrows():
    if row['direction'] == 'down':
      aim = aim + row['value']
    elif row['direction'] == 'up':
      aim = aim - row['value']
    else:
      hp = hp + row['value']
      depth = depth + aim * row['value']
  return hp * depth

In [ ]:
solve_part2(example)

900

In [ ]:
solve_part2(data)

2015547716

## Day 1: Sonar Sweep

In [ ]:
r = requests.get(path+'AoC_01.txt')
# Ignore empty string on the end
data = r.text.split('\n')[:-1]

**Part One**: Count the number of times a depth measurement increases from the previous measurement.

In [ ]:
df = pd.DataFrame([int(d) for d in data], columns=['depth'])
df['diff'] = df.depth.diff()
df

,depth,diff
0,141,NaN
1,140,-1.0
2,160,20.0
3,161,1.0
4,162,1.0
...,...,...
1995,9081,1.0
1996,9084,3.0
1997,9089,5.0
1998,9102,13.0


In [ ]:
df.query('diff > 0').shape

(1692, 2)

**Part Two**: Consider sums of a three-measurement sliding window. Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [ ]:
df.depth.astype('float', copy=False)

0        141.0
1        140.0
2        160.0
3        161.0
4        162.0
         ...  
1995    9081.0
1996    9084.0
1997    9089.0
1998    9102.0
1999    9105.0
Name: depth, Length: 2000, dtype: float64

In [ ]:
df['window'] = df.depth.rolling(3).sum()
df['window_diff'] = df.window.diff()
df.query('window_diff > 0').shape

(1724, 4)